In [1]:
#!/usr/bin/env python

# NMME_PR_prediction.ipynb

In [1]:
'''
    File name: NMME_PR_prediction.ipynb
    Author: Andreas Prein
    E-mail: prein@ucar.edu
    Date created: 04.03.2021
    Date last modified: 04.03.2021

    ##############################################################
    Purpos:

    1) Read in NMME precipitation forecasts in focus catchments

    2) Save monthly catchment average precipitation for future processing

'''

'\n    File name: NMME_PR_prediction.ipynb\n    Author: Andreas Prein\n    E-mail: prein@ucar.edu\n    Date created: 04.03.2021\n    Date last modified: 04.03.2021\n\n    ##############################################################\n    Purpos:\n\n    1) Read in NMME precipitation forecasts in focus catchments\n\n    2) Save monthly catchment average precipitation for future processing\n\n'

In [2]:
from dateutil import rrule
import datetime
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import numpy.ma as ma
import os
from mpl_toolkits import basemap
import pickle
import subprocess
import pandas as pd
from scipy import stats
import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import pylab as plt
import random
import scipy.ndimage as ndimage
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap, cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from pylab import *
import string
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
import shapely.geometry
# import descartes
import shapefile
import math
from scipy.stats.kde import gaussian_kde
from math import radians, cos, sin, asin, sqrt
from scipy import spatial
import scipy.ndimage
import matplotlib.path as mplPath
from scipy.interpolate import interp1d
import time
from math import atan2, degrees, pi
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import csv
import pygrib
from scipy import interpolate
from scipy import signal
# from netcdftime import utime
from scipy.ndimage import gaussian_filter
import scipy.ndimage.filters as filters
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import shapefile as shp

#### speed up interpolation
import scipy.interpolate as spint
import scipy.spatial.qhull as qhull
import numpy as np
import h5py
import xarray as xr

def interp_weights(xy, uv,d=2):
    tri = qhull.Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

In [3]:
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

def MakeShapefile(Regions,
                 LonW,
                 LatW,
                 sShapefiles):
    
    rgrGridCells=[(LonW.ravel()[ii],LatW.ravel()[ii]) for ii in range(len(LonW.ravel()))]
    HUC4_WRF=np.zeros((LonW.shape[0]*LonW.shape[1]))
    for bs in range(len(Regions)):
        Basins = [Regions[bs]]
        for ba in range(len(Basins)):
#             print('        process '+Basins[ba])
            sf = shp.Reader(sShapefiles+Basins[ba])
            df = read_shapefile(sf)
            for sf in range(df.shape[0]):
                ctr = df['coords'][sf]
                if len(ctr) > 10000:
                    ctr=np.array(ctr)[::100,:] # carsen the shapefile accuracy
                else:
                    ctr=np.array(ctr)
                grPRregion=mplPath.Path(ctr)
                TMP=np.array(grPRregion.contains_points(rgrGridCells))
                HUC4_WRF[TMP == 1]=bs+1
    HUC4_WRF=np.reshape(HUC4_WRF, (LatW.shape[0], LatW.shape[1]))
    return HUC4_WRF

In [4]:
########################################
#                            USER INPUT SECTION
mo = 0 # model implemented are [0,1,2]

DataDirERAI='/glade/campaign/mmm/c3we/prein/Projects/Arizona_WTing/data/HandK/'

# Basins
Basins=['AZ_West', 'AZ_East', 'NM_North','NM_South'] 
Region=Basins
sSubregion='/glade/u/home/prein/projects/Arizona_WTing/Shapefiles/'

NMME_dir = '/glade/collections/cdg/data/nmme/output1/'
IFS_dir = '/glade/campaign/mmm/c3we/ECMWF/'
# center/model name, ensemble members, file convention
NMME_models = [['NCAR/CESM1',10,'month_CESM1'], #['CCCMA','NASA-GMAO','NCAR/CESM1', 'NCEP','NOAA-GFDL','UM-RSMAS']
               ['NASA-GMAO/GEOS-5',10,'day_GEOS-5'], # maskes out below surface areas --> use 650 hPa level
               ['UM-RSMAS/CCSM4', 10, 'mon_CCSM4'],
               ['CCCMA/CanCM4', 10, 'Amon_CanCM4'],  # only has 675 hPa data
               ['IFS', 25, 'day_CanCM4']] # only 7 month forecast but 25 members
ConstantFile = [NMME_dir+'NCAR/CESM1/19820101/day/atmos/hus/hus_day_CESM1_19820101_r4i1p1_19820100-19821231.nc4',
               NMME_dir+'NASA-GMAO/GEOS-5/19820101/day/atmos/hus/hus_day_GEOS-5_19820101_r1i1p1.nc',
               NMME_dir+'UM-RSMAS/CCSM4/20050801/day/atmos/hus/hus_day_CCSM4_20050801_r10i1p1_20050801-20060731.nc',
               NMME_dir+'CCCMA/CanCM4/19840101/day/atmos/v20181101/hus/hus_day_CanCM4_198401_r10i1p1_19840101-19841231.nc4',
               IFS_dir+'20050601/Q_GDS0_ISBL/Q_GDS0_ISBL_day_ECMWF_mem01_20050601.nc']
DataDir = [NMME_dir,
          NMME_dir,
          NMME_dir,
          NMME_dir,
          IFS_dir]
# for each variable we have the general varname, the netCDF var name, and the pressure level (-1 means 2D field), netCDF varname
ImputVars=[['PR','precip','precip','mon'],
            ['PR','pr','pr','day'],
            ['PR','precip','PRECIP','mon'],
            ['PR','prlr','prlr','mon'],
            ['PR','prec','TP_GDS0_SFC','day']]

SaveDir='/glade/campaign/mmm/c3we/prein/Projects/Arizona_WTing/data/NMME/PR/'

MONTHS=[6,7,8,9,10] # [1,2,3,4,5,6,7,8,9,10,11,12]
StartMonths=[2,3,4,5,6]

dStartDay=datetime.datetime(int(1982), 1, 1,12)
dStopDay=datetime.datetime(int(2010), 12, 31,12)
rgdTimeDD = pd.date_range(dStartDay, end=dStopDay, freq='d')
rgdTimeMM = pd.date_range(dStartDay, end=dStopDay, freq='m')
rgiYY=np.unique(rgdTimeDD.year)
rgdTimeDD = rgdTimeDD[np.isin(rgdTimeDD.month, MONTHS)]
rgdTimeMM = rgdTimeMM[np.isin(rgdTimeMM.month, MONTHS)]


### Loop over Models and Regions and Calculate Monsoon Season PR in Basins

In [5]:
PR_basins_NMME = np.zeros((len(rgiYY), len(MONTHS), len(StartMonths), len(NMME_models), 25, len(Basins))); PR_basins_NMME[:] = np.nan

# ----- Load ERA data for constraining region ------
ERA_data = '/glade/campaign/mmm/c3we/prein/Projects/Arizona_WTing/data/HandK/ERA-Interim_PRISM_data13514_1502_1982-2018_Q850_JJASO.npz'
DATA = np.load(ERA_data)
rgrLonT = DATA['LonWT']
rgrLatT = DATA['LatWT']

for mo in range(len(NMME_models)):
    print('work on '+str(NMME_models[mo][0]))
    if ImputVars[mo][3] == 'mon':
        FREQ = 'm'
    elif ImputVars[mo][3] == 'day':
        FREQ = 'd'
    # read NMME Grid
    sERAconstantFields='/glade/work/prein/reanalyses/ERA-Interim/ERA_Inerim_stationary-files_75x75.nc'
    # read the ERA-Interim elevation
    ncid=Dataset(sERAconstantFields, mode='r')
    rgrLat75=np.squeeze(ncid.variables['latitude'][:])
    rgrLon75=np.squeeze(ncid.variables['longitude'][:])
    rgrHeight=(np.squeeze(ncid.variables['z'][:]))/9.81
    rgrLSM=(np.squeeze(ncid.variables['lsm'][:]))
    ncid.close()
    rgdTimeDD_Full = pd.date_range(datetime.datetime(int(1979), 1, 1,12), end=datetime.datetime(int(2017), 12, 31,12), freq='d')

    # read NMME coordinates
    ncid=Dataset(ConstantFile[mo], mode='r')
    if 'lon' in list(ncid.variables.keys()):
        rgrLonS=np.squeeze(ncid.variables['lon'][:])
        rgrLatS=np.squeeze(ncid.variables['lat'][:])
    elif 'LON' in list(ncid.variables.keys()):
        rgrLonS=np.squeeze(ncid.variables['LON'][:])
        rgrLatS=np.squeeze(ncid.variables['LAT'][:])
    elif 'g0_lon_4' in list(ncid.variables.keys()):
        rgrLonS=np.squeeze(ncid.variables['g0_lon_4'][:])
        rgrLatS=np.squeeze(ncid.variables['g0_lat_3'][:])
    ncid.close()
    rgrLonSF, rgrLatSF = np.meshgrid(rgrLonS, rgrLatS)
    rgrLonSF[rgrLonSF>180] = rgrLonSF[rgrLonSF>180]-360

    if NMME_models[mo][0] == 'IFS':
        rgrLatSF = rgrLatSF[::-1] # IFS lat runs from N to S

    # get the region of interest
    iAddCells= 10 # grid cells added to subregion
    iWest=np.argmin(np.abs(rgrLonT.min() - rgrLonSF[0,:]))-iAddCells
    iEast=np.argmin(np.abs(rgrLonT.max() - rgrLonSF[0,:]))+iAddCells
    iNort=np.argmin(np.abs(rgrLatT.max() - rgrLatSF[:,0]))+iAddCells
    iSouth=np.argmin(np.abs(rgrLatT.min() - rgrLatSF[:,0]))-iAddCells

    rgrLonS=rgrLonSF[iSouth:iNort,iWest:iEast]
    rgrLatS=rgrLatSF[iSouth:iNort,iWest:iEast]
    
    ### Calculate mask file for subregions
    MASK = MakeShapefile(Basins,
                 rgrLonS,
                 rgrLatS,
                 sSubregion)
        

    for yy in range(len(rgiYY)):
        print('    process '+str(rgiYY[yy]))
        for mm in range(len(StartMonths)):
            print('        month '+str(StartMonths[mm]))
            TIMEstamp = str(rgiYY[yy])+str(StartMonths[mm]).zfill(2)+'01'
            dStartACT=datetime.datetime(rgiYY[yy], StartMonths[mm], 1,0)
            if NMME_models[mo][0] in ['NCAR/CESM1','UM-RSMAS/CCSM4','CCCMA/CanCM4']:
                dStopACT = dStartACT + relativedelta(months=+12) - timedelta(days=1)
                rgdTimeACT = pd.date_range(dStartACT, end=dStopACT, freq=FREQ)
            elif NMME_models[mo][0] == 'IFS':
                dStopACT = dStartACT + relativedelta(months=+7)
                rgdTimeACT = pd.date_range(dStartACT, end=dStopACT, freq=FREQ)
            else:
                dStopACT = dStartACT + relativedelta(months=+9) - timedelta(days=1)
                rgdTimeACT = pd.date_range(dStartACT, end=dStopACT, freq=FREQ)
            
            if NMME_models[mo][0] == 'CCCMA/CanCM4':
                DirNameAct = DataDir[mo]+NMME_models[mo][0]+'/'+TIMEstamp+'/'+ImputVars[mo][3]+'/atmos/v20181101/'+ImputVars[mo][1]+'/'
            elif NMME_models[mo][0] == 'IFS':
                if (rgiYY[yy] >= 1993) & (StartMonths[mm] >= 4):
                    DirNameAct = DataDir[mo]+'/'+TIMEstamp+'/'+ImputVars[mo][1]+'/'
                else:
                    continue
            else:
                DirNameAct = DataDir[mo]+NMME_models[mo][0]+'/'+TIMEstamp+'/'+ImputVars[mo][3]+'/atmos/'+ImputVars[mo][1]+'/'
            
            for en in range(NMME_models[mo][1]):
                YYYYMMDD_start = str(rgdTimeACT[0].year)+str(rgdTimeACT[0].month).zfill(2)+'00' #str(rgdTimeACT[0].day).zfill(2)
                YYYYMMDD_stop = str(rgdTimeACT[-1].year)+str(rgdTimeACT[-1].month).zfill(2)+'*' #str(rgdTimeACT[-1].day).zfill(2)
                if NMME_models[mo][0] == 'CCCMA/CanCM4':
                    TIMEstamp = str(rgiYY[yy])+str(StartMonths[mm]).zfill(2)
                if NMME_models[mo][0] != 'IFS':
                    try:
                        FileName = glob.glob(DirNameAct+ImputVars[mo][1]+'_'+NMME_models[mo][2]+'_'+TIMEstamp+'_r'+str(en+1)+'i1p1*'+'.nc*')[0]
                    except:
                        print('    not found - '+DirNameAct+ImputVars[mo][1]+'_'+NMME_models[mo][2]+'_'+TIMEstamp+'_r'+str(en+1)+'i1p1*'+'.nc*')
                        continue
                else:
                    FileName = glob.glob(DirNameAct+ImputVars[mo][1]+'_day_ECMWF_mem'+str(en+1).zfill(2)+'*.nc')[0]
                iTime = np.isin(rgdTimeACT.month, MONTHS)
                # read the data
                ncid=Dataset(FileName, mode='r')
                if NMME_models[mo][0] == 'IFS':
                    DATAact=np.squeeze(ncid.variables[ImputVars[mo][2]][:,:,iTime,:,:])[:,::-1,:][:,iSouth:iNort,iWest:iEast]
                elif NMME_models[mo][0] == 'NASA-GMAO/GEOS-5':
                    try:
                        DATAact=np.squeeze(ncid.variables[ImputVars[mo][2]][iTime,:,iSouth:iNort,iWest:iEast])
                    except:
                        DATAact=np.squeeze(ncid.variables[ImputVars[mo][2]][iTime[:-1],:,iSouth:iNort,iWest:iEast])
                else:
                    try:
                        DATAact=np.squeeze(ncid.variables[ImputVars[mo][2]][iTime,iSouth:iNort,iWest:iEast])
                    except:
                        # some models do not have leap years
                        DATAact=np.squeeze(ncid.variables[ImputVars[mo][2]][iTime[:-1],iSouth:iNort,iWest:iEast])
                ncid.close()
                
                if NMME_models[mo][0] == 'IFS':
                    # calculate daily precipitation from accumulation
                    PR_IFS = DATAact[1:,:,:]-DATAact[:-1,:,:]
                    PR_IFS = np.append(PR_IFS[0,:,:][None,:,:],PR_IFS, axis=0)
                    DATAact = PR_IFS
                
                # calculate monthly means if data is daily
                if ImputVars[mo][3] == 'day':
                    MON_dat = np.array([np.mean(DATAact[rgdTimeACT[iTime].month == MONTHS[ii],:,:], axis=0) for ii in range(len(MONTHS))])
                    DATAact = MON_dat
                    
                # average monthly precipitation over catchments
                for ba in range(len(Basins)):
                    PR_basins_NMME[:,yy,:,mm,mo,en,ba] = np.mean(DATAact[:,MASK == (ba+1)], axis=(1))

np.savez(SaveDir+'NMME_PR-forecasts_Monsoon-Basins.npz',
        years=rgiYY,
        months = MONTHS,
        ForecastStartMonths = StartMonths,
        models = NMME_models,
        basins = Basins,
        precipitation = PR_basins_NMME,
        precip_dims = 'year,month,start-month,model,ensemble-member,basin')

work on NCAR/CESM1
    process 1982
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1983
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1984
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1985
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1986
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1987
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1988
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1989
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1990
        month 2
        month 3
        month 4
        month 5
        month 6
    process 1991
        month 2
        month 3
        month 4
        month 5
        month 6
    process

IndexError: 
Boolean array must have the same shape as the data along this dimension.

In [8]:
PR_basins_NMME.shape

(29, 5, 5, 5, 25, 4)

In [ ]:
SaveDir+'NMME_PR-forecasts_Monsoon-Basins.npz'

### Load PRISM and calculate basin averages for validation

In [ ]:
# Calculate Mask first
ncid=Dataset('/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_2014.nc', mode='r') # open the netcdf file
rgrLatPR=np.squeeze(ncid.variables['lat'][:])
rgrLonPR=np.squeeze(ncid.variables['lon'][:])
ncid.close()

MASK = MakeShapefile(Basins,
                 rgrLonPR,
                 rgrLatPR,
                 sSubregion)

In [ ]:
# Read the PRISM data
rgrPRdata = np.zeros((len(rgiYY), len(MONTHS), len(Basins))); rgrPRdata[:] = np.nan
for yy in range(len(rgiYY)):
    print('working on '+str(rgiYY[yy]))
    rgdTimeYY = pd.date_range(datetime.datetime(rgiYY[0]+yy, 1, 1,0), end=datetime.datetime(rgiYY[0]+yy, 12, 31,23), freq='d')
    rgiDD=np.where(((rgdTimeYY.year == rgiYY[0]+yy) & (np.isin(rgdTimeYY.month, MONTHS))))[0]
    ncid=Dataset('/glade/campaign/mmm/c3we/prein/observations/PRISM/data/PR/PRISM_daily_ppt_'+str(rgiYY[0]+yy)+'.nc', mode='r') # open the netcdf file
    PR_PRISM_DD = np.squeeze(ncid.variables['PR'][rgiDD,:])
    ncid.close()
    PR_PRISM_MM = np.array([np.mean(PR_PRISM_DD[rgdTimeYY[rgiDD].month == MONTHS[ii],:,:], axis=0) for ii in range(len(MONTHS))])
    # calcualte averages over regions
    for ba in range(len(Basins)):
        rgrPRdata[yy,:,ba] = np.mean(PR_PRISM_MM[:,MASK == (ba+1)], axis=(1))
            

In [126]:
np.savez(SaveDir+'PRISM_PR-observations_Monsoon-Basins.npz',
        years=rgiYY,
        months = MONTHS,
        basins = Basins,
        precipitation = rgrPRdata,
        precip_dims = 'year,month,basin')

In [127]:
SaveDir+'PRISM_PR-observations_Monsoon-Basins.npz'

'/glade/campaign/mmm/c3we/prein/Projects/Arizona_WTing/data/NMME/PR/PRISM_PR-observations_Monsoon-Basins.npz'